In [1]:
from gensim.models import KeyedVectors

In [2]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

In [3]:
vectors_seed1_dict = {}
vectors_seed2_dict = {}
for sub in subcorpora:
    vectors_seed1 = KeyedVectors.load("../data/large_data/vectors_{}_seed1.kv".format(sub))
    vectors_seed1_dict[sub] = vectors_seed1
    vectors_seed2 = KeyedVectors.load("../data/large_data/vectors_{}_seed2.kv".format(sub))
    vectors_seed2_dict[sub] = vectors_seed2

In [5]:
vectors_seed1_dict["archaic"]["ἀγαθός"]

array([ 5.47457278e-01, -5.95615149e-01, -3.66489857e-01, -5.23532331e-01,
        7.57952869e-01, -5.74427173e-02,  5.54002047e-01, -6.15041256e-01,
        1.46978512e-01,  2.94663429e-01,  1.60077661e-01, -2.06675366e-01,
       -4.98863518e-01, -3.22766483e-01, -6.25970483e-01,  1.13165289e-01,
       -1.54542953e-01,  7.05618918e-01, -1.53131306e-01,  5.15999377e-01,
       -3.42287451e-01, -2.20488720e-02,  6.30944729e-01,  1.62394986e-01,
       -1.28771946e-01, -1.15331984e+00,  7.94084787e-01, -6.22678161e-01,
       -2.59171009e-01,  5.42359687e-02, -7.21092448e-02,  2.66373545e-01,
       -1.36334062e-01, -1.58509210e-01, -2.76235700e-01, -7.68341184e-01,
       -2.71129698e-01, -1.08893886e-01, -3.07754785e-01, -1.45118520e-01,
       -1.11648114e-02,  3.05862445e-02,  4.32406783e-01, -2.94452876e-01,
        2.36617580e-01,  1.80239230e-02, -2.16818780e-01,  2.58344352e-01,
        4.71329736e-03, -7.93799981e-02, -6.59710407e-01, -1.15705088e-01,
        1.29746139e-01,  